# Introduction to iFEM

## Geometry of the problem : 

A beam from point A to B simply supported with length L, Youngs modulus E and Inertia I :
                                                                              
         A O___________________________________________________O   B     
          /\                      L                           /\         
         /  \                                                /  \        

nel, the number of elements for spatial discretization

## Convenient imports and tools

In [69]:
using Muscade, StaticArrays, GLMakie, CSV, DataFrames, Interpolations
using Muscade.Toolbox

In [2]:
"""
    save_timeseries_csv(path; comps::Dict, time=nothing)

Save multiple time-series (vector-of-vectors) into a single CSV file.

`comps` should be a Dict mapping a short series name (e.g. "X", "Y", "Z" or "force")
to a vector of length `nsteps`, where each element is a vector of length `nnodes` (or a scalar for node-insensitive series).

The produced CSV has `nsteps` rows (one per time step). If `time` is provided it is saved as the first column `time`.
For each series `name` with `nnodes > 1` the CSV will contain columns `name_1, name_2, ..., name_nnodes` (per-node values).
For scalar series (no underscore in column naming) `name` will be written as a single column.

Returns `true` on success.
"""
function save_timeseries_csv(path; comps::Dict=Dict(), time=nothing)
    isempty(comps) && error("`comps` must be a Dict mapping series names to vector-of-vectors, e.g. Dict(\"X\"=>x_, ...)")

    # Basic validation: all series must have same number of time steps
    nsteps = nothing
    for (name, vecs) in comps
        if nsteps === nothing
            nsteps = length(vecs)
        else
            @assert length(vecs) == nsteps "All series must have same number of time steps; mismatch for $name"
        end
    end

    # Convert each series into a (nnodes, nsteps) matrix where possible
    comp_mats = Dict{String, Matrix{Float64}}()
    comp_scalars = Dict{String, Vector{Float64}}()
    for (name, vecs) in comps
        # Determine if inner elements are vectors (per-node) or scalars
        first_inner = first(vecs)
        if isa(first_inner, AbstractVector)
            nnodes = length(first_inner)
            for (i, vv) in enumerate(vecs)
                @assert length(vv) == nnodes "Inconsistent node length in $name at step $i"
            end
            M = hcat([vec(v) for v in vecs]...)   # (nnodes, nsteps)
            comp_mats[name] = M
        else
            # treat as scalar-per-time series
            comp_scalars[name] = [float(v) for v in vecs]
        end
    end

    base = replace(path, r"\.csv$" => "")
    df = DataFrame()
    if time !== nothing
        @assert length(time) == nsteps "time vector must match number of steps"
        df.time = time
    end

    # Add scalar series first
    for (name, vec) in comp_scalars
        df[!, Symbol(name)] = vec
    end

    # Add per-node columns for each matrix series
    for (name, M) in comp_mats
        nnodes = size(M, 1)
        for node in 1:nnodes
            colname = Symbol(string(name, "_", node))
            df[!, colname] = vec(M[node, :])
        end
    end

    CSV.write(base * ".csv", df)
    return true
end


"""
    load_timeseries_csv(path)

Load CSV file previously written by `save_timeseries_csv`.
- Accepts `path` which can be the base prefix (e.g. "results_combined").
- Returns a NamedTuple with fields:
  - `time`: time vector (if present in CSV), or nothing
  - `series`: Dict mapping series names (e.g., "X", "Y", "Z") to (nnodes, nsteps) matrices
  - `scalar_series`: Dict mapping scalar series names to vectors of length nsteps
  - `shape`: (nnodes, nsteps) tuple for per-node series (or nothing if no per-node series)
"""
function load_timeseries_csv(path)
    base = replace(path, r"\.csv$" => "")
    csvfile = base * ".csv"
    
    !isfile(csvfile) && error("CSV file not found: $csvfile")
    
    df = CSV.File(csvfile) |> DataFrame
    nsteps = size(df, 1)
    
    # Extract time column if present
    time = nothing
    colnames = String.(names(df))
    if "time" in colnames
        time = Vector(df[:, :time])
        colnames = filter(x -> x != "time", colnames)
    end
    
    # Parse column names to identify per-node and scalar series
    # Per-node columns: "X_1", "X_2", ...; Scalar columns: "force", etc.
    series_dict = Dict{String, Matrix{Float64}}()
    scalar_dict = Dict{String, Vector{Float64}}()
    
    per_node_cols = filter(c -> occursin("_", c), colnames)
    scalar_cols = filter(c -> !occursin("_", c), colnames)
    
    # Group per-node columns by series name
    for col in per_node_cols
        parts = split(col, "_")
        if length(parts) == 2
            series_name = parts[1]
            node_idx = parse(Int, parts[2])
            
            if !haskey(series_dict, series_name)
                series_dict[series_name] = zeros(0, nsteps)
            end
        end
    end
    
    # Reconstruct matrices for each per-node series
    for (series_name, _) in series_dict
        cols_for_series = filter(c -> startswith(c, series_name * "_"), per_node_cols)
        nnodes = length(cols_for_series)
        # Sort by numeric index, not lexicographic (to get X_1, X_2, ..., X_10, X_11 not X_1, X_10, X_11, ...)
        cols_sorted = sort(cols_for_series; by=c -> parse(Int, split(c, "_")[2]))
        M = Matrix{Float64}(undef, nnodes, nsteps)
        for (node_idx, col) in enumerate(cols_sorted)
            M[node_idx, :] = vec(Vector(df[:, Symbol(col)]))
        end
        series_dict[series_name] = M
    end
    
    # Load scalar series
    for col in scalar_cols
        scalar_dict[col] = Vector(df[:, Symbol(col)])
    end
    
    # Infer shape from first per-node series (if any)
    shape = nothing
    if !isempty(series_dict)
        first_series = first(values(series_dict))
        shape = size(first_series)
    end
    
    return (time=time, series=series_dict, scalar_series=scalar_dict, shape=shape)
end

load_timeseries_csv

## Model creation

In [3]:
R   = 0.0;          # Radius of the bend [m]
EI₂ = 833.33e3;     # Bending stiffness [Nm²]
EI₃ = 833.33e3;     # Bending stiffness [Nm²]
EA  = 1e8;          # Axial stiffness [N]
GJ  = 705e3;        # Torsional stiffness [Nm²]
L   = 10.;           # Length of the beam [m]

nel         = 20
nnodes      = nel+1
nodeCoord   = hcat( -5. .+ ((1:nnodes).-1)/(nnodes-1)*L,
                     0  .+ zeros(Float64, nnodes, 1),
                     0  .+ zeros(Float64, nnodes, 1))
mat         = BeamCrossSection(EA=EA,EI₂=EI₂,EI₃=EI₃,GJ=GJ,μ=1.,ι₁=1.)

BeamCrossSection(1.0e8, 833330.0, 833330.0, 705000.0, 1.0, 1.0)

In [4]:
function createSimplySupportedBeam(name::Symbol; bPlanar=false)
    model       = Model(name)
    nodid       = addnode!(model, nodeCoord)
    mesh        = hcat(nodid[1:nnodes-1],nodid[2:nnodes])
    eleid       = addelement!(model, EulerBeam3D, mesh;mat=mat, orient2=SVector(0.,1.,0.))
    [addelement!(model,Hold,[nodid[1]]  ;field) for field∈[:t1,:t2,:t3,:r1]];   # Support at one end
    [addelement!(model,Hold,[nodid[nnodes]]  ;field) for field∈[:t1, :t2,:t3,:r1]];      # Support at the other end
    if bPlanar 
        [[addelement!(model,Hold,[nodid[i]] ;field) for field∈[:t3]] for i in 2:nnodes-1] 
    end
    return model, nodid, nnodes, eleid
end

createSimplySupportedBeam (generic function with 1 method)

In [5]:

function createDampedSimplySupportedBeam(name::Symbol, β::Float64; bPlanar=false)
    model       = Model(name)
    nodid       = addnode!(model, nodeCoord)
    mesh        = hcat(nodid[1:nnodes-1],nodid[2:nnodes])
    damped_mat         = DampedBeamCrossSection(EA=EA,EI₂=EI₂,EI₃=EI₃,GJ=GJ,μ=1.,ι₁=1.,β=β)
    eleid       = addelement!(model, EulerBeam3D, mesh;mat=damped_mat, orient2=SVector(0.,1.,0.))
    [addelement!(model,Hold,[nodid[1]]  ;field) for field∈[:t1,:t2,:t3,:r1]];   # Support at one end
    [addelement!(model,Hold,[nodid[nnodes]]  ;field) for field∈[:t1, :t2,:t3,:r1]];      # Support at the other end
    if bPlanar 
        [[addelement!(model,Hold,[nodid[i]] ;field) for field∈[:t3]] for i in 2:nnodes-1] 
    end
    return model, nodid, nnodes
end

createDampedSimplySupportedBeam (generic function with 1 method)

## Load scenarii
### Impulse load

In [ ]:
function impulse_load(A, t, t₀; Δt = 0.05)
    abs(t-t₀)<Δt ? impulse_load = A : impulse_load = 0.
end

function impulse_load(A, t, t₀, T; Δt = 0.05)
    abs(t%T-t₀)<Δt ? impulse_load = A : impulse_load = 0.
end

### Sinusoidal load

In [ ]:
function sinus_load(A, t, T, ϕ)
    sinus_load = A * sin(2*π*t/T + ϕ)
end

function distributed_sinus_load(q, t, T_max, L, x)
    distributed_sinus_load = q*(t/T_max)*sin(2*π*x/L)
end

### Ramp load

In [ ]:
function ramp_load(A, T, t)
    t₁ = T[1]
    t₂ = T[2]
    t < t₁ ? ramp_load = 0.0 :
    t < t₂ ? ramp_load = A*(t-t₁)/(t₂-t₁) :
    ramp_load = A
end

## Static analysis

In [23]:
model_stat, nodid, nnodes   = createSimplySupportedBeam(:StaticAnalysis)

# Static_Loads                = [A for A in -50000.:-50000.:-150000.]
A = -5e6
@functor with(A) constant_load(t) = t >= 100 ? A : A/100*(t+1)
# @functor with(A) constant_load(t) = A
addelement!(model_stat,DofLoad,[nodid[floor(Int,nnodes/2)]];field=:t3,value=constant_load )

initialstate                = initialize!(model_stat);

loadSteps                   = [i for i in 1.:10.:500.];
nLoadSteps                  = length(loadSteps)
state                       = solve(SweepX{0};initialstate=initialstate,time=loadSteps,verbose=true,maxΔx=1e-8, maxiter = 50);




Muscade: SweepX{0} solver

    step   1 converged in  11 iterations. |Δx|=2.4e-09 |Lλ|=1.1e-07
    step   2 converged in   8 iterations. |Δx|=1.1e-09 |Lλ|=1.8e-07
    step   2 converged in   8 iterations. |Δx|=1.1e-09 |Lλ|=1.8e-07
    step   3 converged in   6 iterations. |Δx|=7.0e-10 |Lλ|=1.7e-07
    step   4 converged in   6 iterations. |Δx|=3.9e-09 |Lλ|=1.6e-07
    step   3 converged in   6 iterations. |Δx|=7.0e-10 |Lλ|=1.7e-07
    step   4 converged in   6 iterations. |Δx|=3.9e-09 |Lλ|=1.6e-07
    step   5 converged in   5 iterations. |Δx|=3.3e-09 |Lλ|=1.7e-07
    step   6 converged in   5 iterations. |Δx|=2.3e-09 |Lλ|=2.1e-07
    step   7 converged in   5 iterations. |Δx|=1.7e-09 |Lλ|=1.8e-07
    step   5 converged in   5 iterations. |Δx|=3.3e-09 |Lλ|=1.7e-07
    step   6 converged in   5 iterations. |Δx|=2.3e-09 |Lλ|=2.1e-07
    step   7 converged in   5 iterations. |Δx|=1.7e-09 |Lλ|=1.8e-07
    step   8 converged in   5 iterations. |Δx|=9.7e-10 |Lλ|=1.6e-07
    step   9 conve

In [7]:
x_ = [getdof(state[idxLoad];field=:t1,nodID=nodid[1:nnodes]) for idxLoad ∈ 1:nLoadSteps]
y_ = [getdof(state[idxLoad];field=:t2,nodID=nodid[1:nnodes]) for idxLoad ∈ 1:nLoadSteps]
z_ = [getdof(state[idxLoad];field=:t3,nodID=nodid[1:nnodes]) for idxLoad ∈ 1:nLoadSteps]
r_ = [getdof(state[idxLoad];field=:r3,nodID=nodid[1:nnodes]) for idxLoad ∈ 1:nLoadSteps]

fig     = Figure(size = (1000,1000))
ax      = Axis3(fig[1,1],xlabel="x [m]", ylabel="y [m]", zlabel="z [m]",aspect=:equal, title = "Static analysis results")
clr     = [:black,:blue,:green,:red]
for idxLoad ∈ 1:nLoadSteps
    draw!(ax,state[idxLoad];EulerBeam3D=(;nseg=10, line_color = clr[idxLoad%4+1]))
end
xlims!(ax, -5,5); ylims!(ax, -5,5); zlims!(ax, -5,5);
fig

In [8]:
timeseries = Dict(
    "X" => x_,
    "Y" => y_,
    "Z" => z_,
    "R" => r_
)
save_timeseries_csv("results_combined"; comps=timeseries, time=loadSteps)

true

## Eigenvalue analysis

In [ ]:
model_eig, nodid, nnodes   = createSimplySupportedBeam(:EigAnalysis; bPlanar = true)
[addelement!(model_eig,DofLoad,[nodid[nodeidx]];field=:t2,value=t-> sinus_load(200., t, 10., 0.)) for nodeidx=1:nnodes];

initialstate    = initialize!(model_eig);
state_eig       = solve(SweepX{0};initialstate,time=[0.]);
nmod            = 15
#res             = solve(EigX{ℝ};state=state_eig[1],nmod); Assembly has some issue at this date, will have to look at it a bit more at some point


## Dynamic analysis with various load scenarii
### Sinusoidal nodal load

In [ ]:
model_dyn, nodid, nnodes   = createSimplySupportedBeam(:DynAnalysis_sin)

A, Tp, ϕ = 1e4, 10., 0.
@functor with(A, T, ϕ) sin_load(t) = t == 0.15 ? A * sin(2*π*t/Tp + ϕ) : 0.0
addelement!(model_dyn,DofLoad,[nodid[floor(Int,nnodes/2)]];field=:t2,value= t -> sin_load )

initialstate                = initialize!(model_dyn; time = 0.);
Tsin                        = 1.:0.05:10.
nLoadSteps                  = length(Tsin)
state                       = solve(SweepX{2};initialstate,time=Tsin,verbose=true,maxΔx=1e-8, maxiter = 80);

In [ ]:
x_sin = [[getdof(state[idxLoad];field=:t1,nodID=[nodid[node]]) for idxLoad ∈ 1:nLoadSteps] for node in 1:nnodes]
y_sin = [[getdof(state[idxLoad];field=:t2,nodID=[nodid[node]]) for idxLoad ∈ 1:nLoadSteps] for node in 1:nnodes]
z_sin = [[getdof(state[idxLoad];field=:t3,nodID=[nodid[node]]) for idxLoad ∈ 1:nLoadSteps] for node in 1:nnodes]
r3_sin = [[getdof(state[idxLoad];field=:r3,nodID=[nodid[node]]) for idxLoad ∈ 1:nLoadSteps] for node in 1:nnodes]

figure     = Figure(size = (1000,1000))
ax      = Axis3(figure[1,1],xlabel="x [m]", ylabel="y [m]", zlabel="z [m]",aspect=:equal)
for to_draw in 1:10:nLoadSteps
    draw!(ax,state[to_draw];EulerBeam3D=(;nseg=20,  line_color= RGBf(1.0, to_draw/nLoadSteps, 0.)))
end


In [ ]:
req = @request gp(x∂X₀)
loads = getresult(statew[1500],req,[eleid[1]])
print(loads)

In [ ]:

figure     = Figure(size = (1000,1000))
ax      = Axis3(figure[1,1],xlabel="x [m]", ylabel="y [m]", zlabel="z [m]",aspect=:equal)
rng = 50:5:90 # 1:100:nLoadSteps

for to_draw in rng
    draw!(ax,statew[to_draw];EulerBeam3D=(;nseg=20,  line_color= RGBf((to_draw-rng[1])/length(rng)*0.5, 0.99.-(to_draw-rng[1])/length(rng)*0.5, 0.)))
end
display(figure)
figure

In [ ]:

fig_zt = Figure(size = (1000,1000))
x_ = [getdof(statew[idxLoad];field=:t1,nodID=[nodid[21]]) for idxLoad ∈ 1:nLoadSteps]
y_ = [getdof(statew[idxLoad];field=:t2,nodID=[nodid[10]]) for idxLoad ∈ 1:nLoadSteps]
z_ = [getdof(statew[idxLoad];field=:t3,nodID=[nodid[20]]) for idxLoad ∈ 1:nLoadSteps]
z_m = [getdof(statew[idxLoad];field=:t3,nodID=[nodid[10]]) for idxLoad ∈ 1:nLoadSteps]
ax = Axis(fig_zt[1, 1], xlabel="Time, t [s]", ylabel="Displacement in the y-direction [m]", title = "Direct solution to impulse load at t = 0.15s, load distributed on all nodes, No Damping")
lines!(ax, T, vcat(z_...); label="Node 20")
lines!(ax, T, vcat(z_m...); label="Node 10")
axislegend()


save("no_damping.png",fig_zt)


Local initial load

In [ ]:
model_dyn, nodid, nnodes   = createSimplySupportedBeam(:DynAnalysis_impulselocal)

A = 1000

addelement!(model_dyn,DofLoad,[nodid[floor(Int, nnodes/2)]];field=:t2,value= t -> t == 0.1 ? A : 0.0)

initialstate                = initialize!(model_dyn; time = 0.);

loadSteps                   = [i for i in 0.1:0.005:10.];
nLoadSteps                  = length(loadSteps)
statew                       = solve(SweepX{2};initialstate,time=loadSteps,verbose=true,maxΔx=1e-6, maxiter = 80);

In [ ]:
fig_zt = Figure(size = (1000,1000))
x_ = [getdof(statew[idxLoad];field=:t1,nodID=[nodid[6]]) for idxLoad ∈ 1:nLoadSteps]
y_ = [getdof(statew[idxLoad];field=:t2,nodID=[nodid[6]]) for idxLoad ∈ 1:nLoadSteps]
z_ = [getdof(statew[idxLoad];field=:t3,nodID=[nodid[6]]) for idxLoad ∈ 1:nLoadSteps]
Axis(fig_zt[1, 1])
scatter!(vcat(y_...))

fig_zt

In [ ]:
figure     = Figure(size = (1000,1000))
ax      = Axis3(figure[1,1],xlabel="x [m]", ylabel="y [m]", zlabel="z [m]",aspect=:equal)
for to_draw in [10,11, 12, 13, 14]
    draw!(ax,statew[to_draw];EulerBeam3D=(;nseg=20,  line_color= RGBf(to_draw/1500, 0., 0.)))
end
display(figure)
figure

## Inverse Crime

### Static inverse crime

#### Load datas

In [64]:
data = load_timeseries_csv("results_combined")
idxLoad = 50
x_restored = data.series["X"][:, idxLoad]
y_restored = data.series["Y"][:, idxLoad]
z_restored = data.series["Z"][:, idxLoad]
r3_restored = data.series["R"][:, idxLoad]
time = data.time;
println(size(x_restored))


(21,)


#### Create model and solve

In [ ]:
# inv_model = Model(:inverse_static)
# nodid2      = addnode!(inv_model, nodeCoord2)
# nnodes = size(nodid2, 1)
# mesh        = hcat(nodid2[1:nnodes-1],nodid2[2:nnodes])
# eleid       = addelement!(inv_model, EulerBeam3D, mesh;mat=mat, orient2=SVector(0.,1.,0.))
# [addelement!(inv_model,Hold,[nodid2[1]]  ;field) for field∈[:t1,:t2,:t3,:r1]];   # Support at one end
# [addelement!(inv_model,Hold,[nodid2[nnodes]]  ;field) for field∈[:t1, :t2,:t3,:r1]];      # Support at the other end

inv_model, nodid, nnodes, eleid   = createSimplySupportedBeam(:inverse_static)

# Helper: accept meas as either a scalar or a function of time
get_meas(meas, t) = isa(meas, Function) ? meas(t) : meas

# Define cost functors with signature (x, t, meas) to match Muscade expectations
@functor with() costX(x, t, meas) = 1 * (get_meas(meas, t) - x)^2
@functor with() costXother(x, t, meas) = 1 * (get_meas(meas, t) - x)^2
@functor with() costU(u, t) = 0.0005*(A-u)^2
@functor with() costUother(u, t) = 10*u^2

e5 = [addelement!(inv_model,SingleDofCost,[nodid[node]];class=:X,field=:t1, cost= costXother, costargs=(meas = x_restored[node],) ) for node in 1:nnodes]
e6 = [addelement!(inv_model,SingleDofCost,[nodid[node]];class=:X,field=:t2, cost= costX,      costargs=(meas = y_restored[node],) ) for node in 1:nnodes]
e7 = [addelement!(inv_model,SingleDofCost,[nodid[node]];class=:X,field=:t3, cost= costXother, costargs=(meas = z_restored[node],) ) for node in 1:nnodes]
e7 = [addelement!(inv_model,SingleDofCost,[nodid[node]];class=:X,field=:r3, cost= costXother, costargs=(meas = r3_restored[node],) ) for node in 1:nnodes]
e2 = [addelement!(inv_model,Muscade.SingleUDof,[nodid[node]]; Xfield=:t3, Ufield=:ut3, cost=costUother ) for node in 1:nnodes-1];
e3 = [addelement!(inv_model,Muscade.SingleUDof,[nodid[node]]; Xfield=:t2, Ufield=:ut2, cost=costU ) for node in 1:nnodes-1];
e4 = [addelement!(inv_model,Muscade.SingleUDof,[nodid[node]]; Xfield=:t1, Ufield=:ut1, cost=costUother ) for node in 1:nnodes-1];


# Muscade.describe(inv_model,:dof)
# Muscade.describe(inv_model,:doftyp)
# Muscade.describe(inv_model,:eletyp)

UndefVarError: UndefVarError: `SingleUDof` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

In [66]:
InvSolver        = DirectXUA{0,0,0}   # Solver for the inverse analysis
maxiter     = 50
maxΔx       = 1e-8
maxΔu       = 1e-8
maxΔa       = 1e-8
maxΔλ       = Inf

initialstate = initialize!(inv_model);
# Use an AbstractRange wrapped in a Vector as required by the solver
inv_timeSteps = 500.0:-10.0:500.
nSteps = length(inv_timeSteps)

stateXUA = solve(InvSolver;
    initialstate = [initialstate],
    time = [inv_timeSteps],
    verbose = true,
    maxiter = maxiter,
    maxΔx = maxΔx,
    maxΔλ = maxΔλ,
    maxΔu = maxΔu,
    maxΔa = maxΔa,
);





Muscade: DirectXUA{0, 0, 0} solver


    Preparing assembler

    Iteration   1
        Assembling, solving, decrementing.
        maxₜ(|ΔΛ|)=2.7e+01 ≤     Inf  
        maxₜ(|ΔX|)=0.0e+00 ≤ 1.0e-08  
        maxₜ(|ΔU|)=2.2e+07 ≤ 1.0e-08  

    Iteration   2
        Assembling, solving, decrementing.
        maxₜ(|ΔΛ|)=2.6e-03 ≤     Inf  
        maxₜ(|ΔX|)=0.0e+00 ≤ 1.0e-08  
        maxₜ(|ΔU|)=0.0e+00 ≤ 1.0e-08  

    Converged in   2 iterations.
    nel=172, nvar=328, nstep=1
    DirectXUA{0, 0, 0} time:  485 [ms]
Muscade done.



    Iteration   1
        Assembling, solving, decrementing.
        maxₜ(|ΔΛ|)=2.7e+01 ≤     Inf  
        maxₜ(|ΔX|)=0.0e+00 ≤ 1.0e-08  
        maxₜ(|ΔU|)=2.2e+07 ≤ 1.0e-08  

    Iteration   2
        Assembling, solving, decrementing.
        maxₜ(|ΔΛ|)=2.6e-03 ≤     Inf  
        maxₜ(|ΔX|)=0.0e+00 ≤ 1.0e-08  
        maxₜ(|ΔU|)=0.0e+00 ≤ 1.0e-08  

    Converged in   2 iterations.
    nel=172, nvar=328, nstep=1
    DirectXUA{0, 0, 0} time:  485 [

### Extracting results

In [67]:
idx = 1

# displacements
println("\n== Displacements :")
x_ = [getdof(stateXUA[idx];field=:t1,nodID=nodid[1:nnodes]) for idx ∈ 1:nSteps]
y_ = [getdof(stateXUA[idx];field=:t2,nodID=nodid[1:nnodes]) for idx ∈ 1:nSteps]
z_ = [getdof(stateXUA[idx];field=:t3,nodID=nodid[1:nnodes]) for idx ∈ 1:nSteps]
r_ = [getdof(stateXUA[idx];field=:r3,nodID=nodid[1:nnodes]) for idx ∈ 1:nSteps];

println(x_)
println(y_)
println(z_)
println(r_)

# Axial force
println("\n== Axial force :")
req = @request gp(resultants(fᵢ))
out = getresult(stateXUA[idx],req,eleid)
Fgp1_ = [ out[idxEl].gp[1][:resultants][:fᵢ] for idxEl ∈ 1:nel]
Fgp2_ = [ out[idxEl].gp[2][:resultants][:fᵢ] for idxEl ∈ 1:nel]
Fgp3_ = [ out[idxEl].gp[3][:resultants][:fᵢ] for idxEl ∈ 1:nel]
Fgp4_ = [ out[idxEl].gp[4][:resultants][:fᵢ] for idxEl ∈ 1:nel];

println(Fgp1_)
println(Fgp2_)
println(Fgp3_)
println(Fgp4_)

# Bending moments
println("\n== Bending moment :")
req = @request gp(resultants(mᵢ))
out = getresult(stateXUA[idx],req,eleid)
Mgp1_ = [ out[idxEl].gp[1][:resultants][:mᵢ] for idxEl ∈ 1:nel]
Mgp2_ = [ out[idxEl].gp[2][:resultants][:mᵢ] for idxEl ∈ 1:nel]
Mgp3_ = [ out[idxEl].gp[3][:resultants][:mᵢ] for idxEl ∈ 1:nel]
Mgp4_ = [ out[idxEl].gp[4][:resultants][:mᵢ] for idxEl ∈ 1:nel];

println(Mgp1_)
println(Mgp2_)
println(Mgp3_)
println(Mgp4_)


== Displacements :
[[0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0;;]]
[[0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0;;]]
[[0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0;;]]
[[0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0;;]]

== Axial force :
[[0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0;;]]
[[0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0;;]]
[[0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0;;]]
[[0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0;;]]

== Axial force :
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0

In [52]:
# Plot axial force resultants (Fgp1_ from previous extraction cell)
using GLMakie
fig = Figure()
ax = Axis(fig[1,1], xlabel="Element index", ylabel="Axial force fᵢ", title="Axial force along elements (gp1)")
lines!(ax, 1:length(Fgp1_), Fgp1_; linewidth=2)
scatter!(ax, 1:length(Fgp1_), Fgp1_; markersize=5)
fig

In [54]:
# Plot bending moments - multiple approaches
using GLMakie, LinearAlgebra

# Approach 1: Magnitude of moment vectors at Gauss point 1
Mgp1_magnitude = [norm(m) for m in Mgp1_]

fig = Figure(size=(1200, 800))

# Subplot 1: Magnitude of M at gp1
ax1 = Axis(fig[1,1], xlabel="Element index", ylabel="Magnitude |mᵢ|", title="Bending moment magnitude at gp1")
lines!(ax1, 1:length(Mgp1_magnitude), Mgp1_magnitude; linewidth=2, label="gp1")
scatter!(ax1, 1:length(Mgp1_magnitude), Mgp1_magnitude; markersize=5)


# Subplot 2: Compare magnitudes across Gauss points
Mgp2_magnitude = [norm(m) for m in Mgp2_]
Mgp3_magnitude = [norm(m) for m in Mgp3_]
Mgp4_magnitude = [norm(m) for m in Mgp4_]

ax2 = Axis(fig[1,2], xlabel="Element index", ylabel="Magnitude |mᵢ|", title="Bending moment magnitude - all Gauss points")
lines!(ax2, 1:length(Mgp1_magnitude), Mgp1_magnitude; label="gp1", linewidth=2)
lines!(ax2, 1:length(Mgp2_magnitude), Mgp2_magnitude; label="gp2", linewidth=2)
lines!(ax2, 1:length(Mgp3_magnitude), Mgp3_magnitude; label="gp3", linewidth=2)
lines!(ax2, 1:length(Mgp4_magnitude), Mgp4_magnitude; label="gp4", linewidth=2)


# Subplot 3: Individual components of moment at gp1 (z-component most relevant for bending)
Mgp1_z = [m[3] for m in Mgp1_]  # Extract z-component (bending about z-axis)

ax3 = Axis(fig[2,1], xlabel="Element index", ylabel="mᵢ,z component", title="Bending moment z-component at gp1")
lines!(ax3, 1:length(Mgp1_z), Mgp1_z; linewidth=2, color=:red)
scatter!(ax3, 1:length(Mgp1_z), Mgp1_z; markersize=5, color=:red)


# Subplot 4: All three components at gp1
Mgp1_x = [m[1] for m in Mgp1_]
Mgp1_y = [m[2] for m in Mgp1_]

ax4 = Axis(fig[2,2], xlabel="Element index", ylabel="Component value", title="All moment components at gp1")
lines!(ax4, 1:length(Mgp1_x), Mgp1_x; label="mᵢ,x", linewidth=2)
lines!(ax4, 1:length(Mgp1_y), Mgp1_y; label="mᵢ,y", linewidth=2)
lines!(ax4, 1:length(Mgp1_z), Mgp1_z; label="mᵢ,z", linewidth=2)


fig
